# Create Agent

### Getting Started

This sample demonstrates how to evaluate Azure AI Agent
Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) *PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) *MODEL_DEPLOYMENT_NAME** - The deployment name of the AI model, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) *AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) *AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) *AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) *AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) *PROJECT_NAME** - Azure AI Project Name
8) *RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name


### Initializing Project Client

In [1]:
import os
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional
import pandas as pd
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
# from user_functions import user_functions
from dotenv import load_dotenv

load_dotenv("../../.env")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant PrP"

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location: The location to fetch weather for.
    :return: Weather information as a JSON string.
    """
    # Mock weather data for demonstration purposes
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    return json.dumps({"weather": weather})

# Adding Tools to be used by Agent 

# Define user functions
user_functions = {fetch_weather}



### Create Agent

In [2]:
# Initialize the FunctionTool with user-defined functions
functions = FunctionTool(functions=user_functions)

agent = project_client.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    tools=functions.definitions,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_aQLKnojDOVD0Kgxgbxg1SNI7


### Create Thread

In [3]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_Ov5W2jmELiSVVLL96zXU0Xgk


## Conversation with Agent
Use below cells to have conversation with the agent
- `Create Message[1]`
- `Execute[2]`

### Create Message[1]

In [4]:
# Create message to thread

MESSAGE = "Can you tell the weather for Seattle?"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_DlOS1txBZAPd04AWN2ClExIX


### Execute[2]

In [5]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

ERROR:root:Error executing function 'fetch_weather': Function 'fetch_weather' not found.


Run finished with status: RunStatus.COMPLETED
Run ID: run_P8lcg2jb5gHPeqQeqTPsvqLr


### List Messages

In [6]:
for message in project_client.agents.list_messages(thread.id, order="asc").data:
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Can you tell the weather for Seattle?
----------------------------------------
Role: MessageRole.AGENT
Content: It seems I'm unable to fetch live weather information for Seattle currently. However, you can check Seattle's weather using popular weather services like Weather.com, AccuWeather, or the app your phone provides. Let me know if you'd like assistance with anything else!
----------------------------------------


# Evaluate

### Get data from agent

In [ ]:
import json
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id

converted_data = converter.convert(thread_id, run_id)
print(json.dumps(converted_data, indent=4))

In [ ]:
# Save the converted data to a JSONL file

file_name = "evaluation_data.jsonl"
evaluation_data = converter.prepare_evaluation_data(thread_ids=thread.id, filename=file_name)


### Setting up evaluator

In [ ]:
from azure.ai.evaluation import ToolCallAccuracyEvaluator , AzureOpenAIModelConfiguration, IntentResolutionEvaluator, TaskAdherenceEvaluator, ViolenceEvaluator
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
)
# Needed to use content safety evaluators
azure_ai_project={
    "subscription_id": os.environ["SUBSCRIPTION_ID"],
    "project_name": os.environ["PROJECT_NAME"],
    "resource_group_name": os.environ["RG_NAME"],
}

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)
intent_resolution = IntentResolutionEvaluator(model_config=model_config)
task_adherence = TaskAdherenceEvaluator(model_config=model_config)

In [ ]:
# Evaluating query and response as strings
# A positive example. Intent is identified and understood and the response correctly resolves user intent
result = intent_resolution(
    query="What are the opening hours of the Eiffel Tower?",
    response="Opening hours of the Eiffel Tower are 9:00 AM to 11:00 PM.",
)
result

In [ ]:
from azure.ai.evaluation import ResponseCompletenessEvaluator

response_completeness = ResponseCompletenessEvaluator(model_config=model_config)
# A negative example. Only half of the statements in the response were complete according to the ground truth  
result = response_completeness(
    response="Itinery: Day 1 take a train to visit Disneyland outside of the city; Day 2 rests in hotel.",
    ground_truth="Itinery: Day 1 take a train to visit the downtown area for city sightseeing; Day 2 rests in hotel."
)
result

In [ ]:
query = "How is the weather in Seattle?"
tool_calls = [{
                    "type": "tool_call",
                    "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
                    "name": "fetch_weather",
                    "arguments": {
                        "location": "Seattle"
                    }
            },
            {
                    "type": "tool_call",
                    "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
                    "name": "fetch_weather",
                    "arguments": {
                        "location": "London"
                    }
            }
            ]

tool_definition = {
                    "name": "fetch_weather",
                    "description": "Fetches the weather information for the specified location.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The location to fetch weather for."
                            }
                        }
                    }
                }
response = tool_call_accuracy(query=query, tool_calls=tool_calls, tool_definitions=tool_definition)
response

### Run Evaluator

In [ ]:
from azure.ai.evaluation import evaluate

# Regenerate the evaluation data file to ensure it is valid
file_name = "evaluation_data.jsonl"
evaluation_data = converter.prepare_evaluation_data(thread_ids=thread.id, filename=file_name)

response = evaluate(
    data=file_name,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project={
        "subscription_id": os.environ["SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RG_NAME"],
    }
)
pprint(f'AI Foundary URL: {response.get("studio_url")}')

In [ ]:
response = evaluate(
    data="./sample_synthetic_conversations.jsonl",
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
        "response_completeness": response_completeness,
    },
    azure_ai_project={
        "subscription_id": os.environ["SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RG_NAME"],
    },
)
response.get("studio_url")


In [ ]:
pprint(response)